Formula 1

In [251]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
import matplotlib.pyplot as plt

# 1 Gather Data
We want to...

In [252]:
data_dir_path = "../data/"
# Read the csv files
results = pd.read_csv(data_dir_path + "results.csv", na_values=np.nan)
driver_standings = pd.read_csv(data_dir_path + "driver_standings.csv", na_values=np.nan)
constructor_standings = pd.read_csv(
    data_dir_path + "constructor_standings.csv", na_values=np.nan
)
raceYears = pd.read_csv(data_dir_path + "races.csv", na_values=np.nan)
names = pd.read_csv(data_dir_path + "drivers.csv", na_values=np.nan)

# Filter columns
raceYears = raceYears[["raceId", "year"]]
# raceYears = raceYears[raceYears["year"] >= 2010]  # Only data after 2010
results = results[["raceId", "driverId", "constructorId", "grid", "position"]]
driver_standings = driver_standings[["raceId", "driverId", "position"]]
constructor_standings = constructor_standings[["raceId", "constructorId", "position"]]
names = names[["driverId", "driverRef"]]

# rename because other csv also have position
driver_standings = driver_standings.rename(columns={"position": "driverStanding"})
constructor_standings = constructor_standings.rename(
    columns={"position": "constructorStanding"}
)

year_driver_standing = pd.merge(raceYears, driver_standings, on=["raceId"], how="inner")

results_driver_standings = pd.merge(
    results, year_driver_standing, on=["raceId", "driverId"], how="inner"
)

joined_data = pd.merge(
    results_driver_standings,
    constructor_standings,
    on=["raceId", "constructorId"],
    how="inner",
)

joined_data = pd.merge(joined_data, names, on=["driverId"], how="inner")
joined_data.drop(columns=["raceId", "constructorId", "driverId"], inplace=True)
joined_data.replace(to_replace="\\N", value=20, inplace=True)
joined_data["position"] = joined_data["position"].astype(int)


# Divide by train data and test data
train_data = joined_data.loc[joined_data["year"] <= 2000]
test_data = joined_data.loc[joined_data["year"] > 2000]

# Drop year before calculating the mean for every driver
train_data.drop("year", axis=1)
test_data.drop("year", axis=1)

# Calculate mean values for every driver
# train_data = train_data.groupby(["driverRef"]).mean()
# test_data = test_data.groupby(["driverRef"]).mean()

X_train = train_data[["grid", "driverStanding", "constructorStanding"]]
Y_train = train_data[["position"]]

X_test = train_data[["grid", "driverStanding", "constructorStanding"]]
Y_test = train_data[["position"]]

In [253]:
def edit_Y(y, regression=True):
    """Edits Y to be 1 or not 1 for regression or not regression.

    Notes:
        Pandas DF is passed by reference, so no returns.

    Args:
        y (pd.DataFrame): Dataframe with column position
        regression (bool, optional): True if regression, false for others.
    """
    y.loc[(y["position"] != 1) & (y["position"] != 2) & (y["position"] != 3)] = (
        0 if regression else -1
    )

In [254]:
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()

edit_Y(Y_train, regression=True)
edit_Y(Y_test, regression=True)

Y_train = Y_train.to_numpy()
Y_test = Y_test.to_numpy()

Y_train = Y_train.reshape(Y_train.shape[0])
Y_test = Y_test.reshape(Y_test.shape[0])

/var/folders/lp/5zshsvbx70v_fwm1wkjj8cww0000gn/T/ipykernel_9311/949196391.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y.loc[(y["position"] != 1) & (y["position"] != 2) & (y["position"] != 3)] = (


# 2 Standardise and Scale

In [255]:
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 3 Regression

In [256]:
def polynomial_logistic_regression(X_train, Y_train, X_test, Y_test, max_degree=6):
    """Polynomial logistic regression

    Args:
        X_train (np.array): Training data
        Y_train (np.array): Training labels
        X_test (np.array): Testing data
        Y_test (np.array): Testing labels
    """
    logreg = LogisticRegression(multi_class="multinomial", max_iter=10_000)
    training_error, testing_error = [], []

    print("Polynomial Logistic Regression Score")
    print("Degree \t Test Score \t Train Score")
    for i in range(1, max_degree + 1):
        # Init polynomial features
        poly = PolynomialFeatures(degree=i)

        # Polynomial fit of training data
        X_train_poly = poly.fit_transform(X_train)

        # Train model
        logreg.fit(X_train_poly, Y_train)

        # Polynomial fit of test data
        X_test_poly = poly.fit_transform(X_test)

        # Get scores
        training_score = logreg.score(X_train_poly, Y_train)
        testing_score = logreg.score(X_test_poly, Y_test)

        # Append errors
        training_error.append(1 - training_score * 100)
        testing_error.append(1 - testing_score * 100)

        # Fancy print B)
        print(f"{i} \t {testing_score*100.0:.2f}% \t {training_score*100.0:.2f}%")

In [257]:
polynomial_logistic_regression(X_train, Y_train, X_test, Y_test, max_degree=11)

Polynomial Logistic Regression Score
Degree 	 Test Score 	 Train Score
1 	 88.95% 	 88.95%
2 	 88.97% 	 88.97%
3 	 88.98% 	 88.98%
4 	 88.99% 	 88.99%
5 	 89.03% 	 89.03%
6 	 89.00% 	 89.00%
7 	 88.97% 	 88.97%
8 	 89.01% 	 89.01%
9 	 89.05% 	 89.05%
10 	 89.07% 	 89.07%
11 	 89.05% 	 89.05%
